Bank marketing dateset taken from UCI  
Link : https://archive.ics.uci.edu/dataset/222/bank+marketing  


The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed. 
The classification goal is to predict if the client will subscribe (yes/no) a term deposit (variable y).

